In [2]:
import numpy as np
from scipy.io import mmread

In [9]:
def is_diagonally_dominant(A):
    m, n = np.shape(A)
    for i in range(n):
        sum_non_diag = 0
        for j in range(n):
            if i != j:
                sum_non_diag += abs(A[i,j])
        if sum_non_diag > abs(A[i,i]):
            return False
    return True

In [10]:
def is_symmetric(A):
    m, n = np.shape(A)
    if m != n:
        return False
    for i in range(n):
        for j in range(i+1, n):
            if A[i,j] != A[j,i]:
                return False
    return True

In [11]:
def is_symmetric_positive_definite(A):
    m, n = np.shape(A)

    # check if A is symmetric
    if not is_symmetric(A):
        return False
    
    # check if all eigenvalues are positive
    eigvals = np.linalg.eigvals(A)
    if np.all(eigvals > 0):
        return True
    else:
        return False

In [12]:
def jacobi(A, b, tolerance=1e-6, max_iterations=10000):

    # find the shape of matrix A
    m, n = np.shape(A)

    # check if A is a square matrix
    if m != n:
        raise ValueError('Matrix A must be square')
    
    # check if A is diagonally dominant
    if not is_diagonally_dominant(A):
        print('Warning: Matrix A is not diagonally dominant. May not converge.')

    # initial guess
    x = np.zeros(n)

    # new guess
    x_new = np.zeros(n)

    # iteration counter
    iteration = 0

    while True:

        for i in range(n):
            summation_term = 0.0
            for j in range(n):
                if i != j:
                    summation_term += A[i, j] * x[j]
            x_new[i] = (b[i] - summation_term) / A[i, i]

        # check for convergence
        error = np.linalg.norm(x - x_new, 2)

        # check for NaN and inf errors and stop if found
        if np.isnan(error) or np.isinf(error):
            raise ValueError(f"Diverged. Error became very high at iteration {iteration}.")
        
        # print('Iteration: ', iteration, 'Error: ', error)
        x = np.copy(x_new)
        iteration += 1

        if iteration == max_iterations:
            raise ValueError("Maximum iterations reached without convergence.")
        
        if error<tolerance:
            # print("Converged!!!")
            break

    return x_new


In [13]:
def gauss_siedel(A, b, tolerance=1e-6, max_iterations=10000):

    # find the shape of matrix A
    m, n = np.shape(A)

    # check if A is a square matrix
    if m != n:
        raise ValueError('Matrix A must be square.')
    
    # check if A is diagonally dominant
    if not is_diagonally_dominant(A):
        #check if A is symmetric positive definite
        if not is_symmetric_positive_definite(A):
            print('Warning: Matrix A is not diagonally dominant or symmetric positive definite. May not converge.')

    # initial guess
    x = np.zeros(n)

    # iteration counter
    iteration = 0

    # loop until error is less than tolerance
    while True:

        x_old = np.copy(x) # to check for convergence

        for i in range(n):
            summation_term = 0
            for j in range(n):
                if i != j:
                    summation_term += A[i,j]*x[j]
            x[i] = (b[i] - summation_term)/A[i,i]

        # check for convergence
        error = np.linalg.norm(x - x_old, 2)

        # check for NaN and inf error and stop if found
        if np.isnan(error) or np.isinf(error):
            raise ValueError(f"Diverged. Error became very high at iteration {iteration}.")
        
        # print('Iteration: ', iteration, 'Error: ', error)
        iteration += 1

        if iteration == max_iterations:
            raise ValueError("Maximum iterations reached without convergence.")
        
        if error<tolerance:
            # print("Converged!!!")
            break
        
    return x

In [14]:
def cholesky(A, b):

   # find the shape of matrix A 
    m, n = np.shape(A)

    # check if A is symmetric positive definite
    if not is_symmetric_positive_definite(A):
        raise ValueError('Matrix A must be symmetric positive definite.')

    # initialize L
    L = np.zeros((n,n))

    # find the Cholesky decomposition
    for i in range(n):
        for j in range(i+1):
            if i == j:
                summation_term = 0
                for k in range(i):
                    summation_term += L[i,k]**2
                L[i,i] = np.sqrt(A[i,i] - summation_term)
            else:
                summation_term = 0
                for k in range(j):
                    summation_term += L[j,k]*L[i,k]
                L[i,j] = (A[i,j] - summation_term)/L[j,j]
    
    # now solving for x
        # Ax = b; LL'x = b
        # Let y = L'x; Ly = b; L'x = y

    # solving Ly = b (using forward substitution)
    y = np.zeros(n)
    for i in range(n):
        summation_term = 0
        for j in range(i):
            summation_term += L[i,j]*y[j]
        y[i] = (b[i] - summation_term)/L[i,i]

    # solving L'x = y (using backward substitution)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        summation_term = 0
        for j in range(i+1, n):
            summation_term += L[j,i]*x[j]
        x[i] = (y[i] - summation_term)/L[i,i]

    return x

In [7]:
import time
import statistics

def statistical_test(A, solver, num_samples=100):
    
    # Generate random true x values
    x_true_list = [np.random.rand(A.shape[0]) for _ in range(num_samples)]
    run_times = []

    for i in range(len(x_true_list)):

        x_true = x_true_list[i]
        print("Sample: ", i)
        b = A.dot(x_true)
        start_time = time.time()
        x = solver(A, b)
        end_time = time.time()
        
        run_time = end_time - start_time
        run_times.append(run_time)
    
    # statistical analysis
    mean_run_time = statistics.mean(run_times)
    std_dev_run_time = statistics.stdev(run_times)
    median_run_time = statistics.median(run_times)

    return mean_run_time, std_dev_run_time, median_run_time

In [11]:
mcca = mmread('/home/subarna/ist_597_phyics_informed_machine_learning/hw1/spd2.mtx')
A = np.real(mcca.toarray())

mean_cholesky, std_dev_cholesky, median_cholesky = statistical_test(A, cholesky)

print("Mean run time for Cholesky: ", mean_cholesky)
print("Standard deviation of run time for Cholesky: ", std_dev_cholesky)
print("Median run time for Cholesky: ", median_cholesky)

Sample:  0
Sample:  1
Sample:  2
Sample:  3
Sample:  4
Sample:  5
Sample:  6
Sample:  7
Sample:  8
Sample:  9
Sample:  10
Sample:  11
Sample:  12
Sample:  13
Sample:  14
Sample:  15
Sample:  16
Sample:  17
Sample:  18
Sample:  19
Sample:  20
Sample:  21
Sample:  22
Sample:  23
Sample:  24
Sample:  25
Sample:  26
Sample:  27
Sample:  28
Sample:  29
Sample:  30
Sample:  31
Sample:  32
Sample:  33
Sample:  34
Sample:  35
Sample:  36
Sample:  37
Sample:  38
Sample:  39
Sample:  40
Sample:  41
Sample:  42
Sample:  43
Sample:  44
Sample:  45
Sample:  46
Sample:  47
Sample:  48
Sample:  49
Sample:  50
Sample:  51
Sample:  52
Sample:  53
Sample:  54
Sample:  55
Sample:  56
Sample:  57
Sample:  58
Sample:  59
Sample:  60
Sample:  61
Sample:  62
Sample:  63
Sample:  64
Sample:  65
Sample:  66
Sample:  67
Sample:  68
Sample:  69
Sample:  70
Sample:  71
Sample:  72
Sample:  73
Sample:  74
Sample:  75
Sample:  76
Sample:  77
Sample:  78
Sample:  79
Sample:  80
Sample:  81
Sample:  82
Sample:  83
Sa

In [6]:
mcca = mmread('/home/subarna/ist_597_phyics_informed_machine_learning/hw1/spd2.mtx')
A = np.real(mcca.toarray())

mean_jacobi, std_dev_jacobi, median_jacobi = statistical_test(A, jacobi)

print("Mean run time for Jacobi: ", mean_jacobi)
print("Standard deviation of run time for Jacobi: ", std_dev_jacobi)
print("Median run time for Jacobi: ", median_jacobi)

NameError: name 'jacobi' is not defined

In [15]:
mcca = mmread('/home/subarna/ist_597_phyics_informed_machine_learning/hw1/spd2.mtx')
A = np.real(mcca.toarray())

mean_gs, std_dev_gs, median_gs = statistical_test(A, gauss_siedel)

print("Mean run time for Gauss-Siedel: ", mean_gs)
print("Standard deviation of run time for Gauss-Siedel: ", std_dev_gs)
print("Median run time for Gauss-Siedel: ", median_gs)

Sample:  0


KeyboardInterrupt: 